In [ ]:
# %% 

import pandas as pd 
import numpy as np 
from sklearn.ensemble import RandomForestRegressor  
from sklearn.pipeline import Pipeline
from pre_processing import importar_dados, preprocessar_dados
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from scipy.stats import randint
import joblib

In [ ]:
# %% 

# Utilizando as funções criadas para importar e pré-processar os dados
# A função 'importar_dados' divide os dados em treino, validação e teste.
# A função 'preprocessar_dados' aplica transformações (como escalonamento e codificação) nos dados.
X_train, X_val, X_test, y_train, y_val, y_test = importar_dados() 
preprocessor = preprocessar_dados()

   ANO_VENCIMENTO  MES_VENCIMENTO  TRIMESTRE  VALOR_FATURA_lag1  \
0            2023               8          3             115.00   
1            2023               4          2            8024.27   
2            2022               4          2              55.20   
3            2024               3          1              69.89   
4            2023               6          2            5430.86   

   VALOR_FATURA_lag2  
0             448.35  
1            4845.25  
2              55.20  
3              68.00  
4              53.04  


In [ ]:
# %%

# Criando o modelo de regressão RandomForestRegressor
# O 'random_state=42' garante que os resultados sejam reprodutíveis.
model = RandomForestRegressor(random_state=42)

In [ ]:
# %% 

# Pipeline
# O pipeline permite integrar o pré-processamento e o modelo em um único fluxo de trabalho.
# Isso facilita a execução do pré-processamento e a aplicação do modelo de forma sequencial e correta.
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Primeiro, aplica o pré-processamento aos dados.
    ('regressor', model)  # Depois, treina o modelo RandomForest.
])

In [ ]:
# %% 

# Definindo os parâmetros a serem testados durante a busca de hiperparâmetros.
# Estamos utilizando uma distribuição aleatória para parâmetros como 'n_estimators' e 'max_depth'.
# Espaço de busca para os hiperparâmetros do RandomForest
param_distributions = {
    'regressor__max_depth': [10, 20, 30, None],
    'regressor__n_estimators': [100, 200, 300],
    'regressor__min_samples_split': [2, 5, 10]
}

# RandomizedSearchCV realiza uma busca aleatória nos parâmetros definidos acima.
# Ele testa combinações aleatórias e avalia qual combinação tem o melhor desempenho.
# Realiza a busca de hiperparâmetros
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions,
    n_iter=10,  # Número de combinações aleatórias a serem testadas
    cv=5,  # Número de folds para validação cruzada
    random_state=42
)

# Realiza a busca de hiperparâmetros ajustando o modelo aos dados de treinamento.
random_search.fit(X_train, y_train)
# Exibe os melhores parâmetros encontrados durante a busca.
print("Melhores hiperparâmetros:", random_search.best_params_)

c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: 

Melhores hiperparâmetros: {'regressor__n_estimators': 300, 'regressor__min_samples_split': 10, 'regressor__max_depth': 10}


In [ ]:
# %% 

# Após a busca de hiperparâmetros, obtemos o modelo final treinado com os melhores parâmetros.
random_forest = random_search.best_estimator_

# %% [markdown]

 ### Avaliando Resultado dos modelos

In [ ]:
#%%

# Avalia o modelo com a validação cruzada
validacao_cruzada = KFold(n_splits=10, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(random_forest, X_val, y_val, cv=validacao_cruzada)
acuracia_media_rf = cross_val_scores.mean()
print(cross_val_scores)
print("Acurácia média do Random Forest:", acuracia_media_rf)

# Faz previsões com o pipeline ajustado
y_pred = random_forest.predict(X_val)

c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\maype\Desktop\projetos\water_forecast\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: 

[0.70068211 0.82462153 0.75077162 0.78959344 0.80598062 0.6827257
 0.53178686 0.81432326 0.87554316 0.83556798]
Acurácia média do Random Forest: 0.7611596289414397


# %% [markdown]

 1. Mean Absolute Error (MAE)
 O MAE calcula o erro médio absoluto entre as previsões e os valores reais. Ele é útil para entender o erro médio sem considerar a direção (positivo ou negativo).

In [ ]:
#%%

from sklearn.metrics import mean_absolute_error

# y_val: valores reais, y_pred: valores previstos
mae = mean_absolute_error(y_val, y_pred)

# Exibindo o valor do MAE
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 781.3485227696502


# %% [markdown]

 2. Mean Squared Error (MSE)
 O MSE calcula o erro quadrado médio. Ele penaliza mais os erros grandes, pois a diferença entre o valor real e o previsto é elevada ao quadrado

In [ ]:
#%%

from sklearn.metrics import mean_squared_error

# y_val: valores reais, y_pred: valores previstos
mse = mean_squared_error(y_val, y_pred)

# Exibindo o valor do MSE
print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 10963749.006984863


# %% [markdown]

 3. Root Mean Squared Error (RMSE)
 O RMSE é a raiz quadrada do MSE. Ele traz a medida do erro para a mesma escala dos dados originais, o que torna mais fácil de interpretar.

In [ ]:
#%%

import numpy as np
# Calculando o RMSE como a raiz quadrada do MSE
rmse = np.sqrt(mse)

# Exibindo o valor do RMSE
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 3311.155237524339


# %% [markdown]

 4. Coefficient of Determination (R²)
 O R² (ou coeficiente de determinação) mede a proporção da variação nos dados que o modelo é capaz de explicar. Um valor de 1 significa explicação perfeita, enquanto um valor de 0 significa que o modelo não explicou nada além da média.

In [ ]:
#%%

from sklearn.metrics import r2_score

# y_val: valores reais, y_pred: valores previstos
r2 = r2_score(y_val, y_pred)

# Exibindo o valor do R²
print(f"Coefficient of Determination (R²): {r2}")

Coefficient of Determination (R²): 0.7442861960012085


# %% [markdown]

 5. Explained Variance Score
 A Expained Variance Score calcula a variação explicada pelo modelo. Um valor próximo de 1 significa que o modelo explicou a maior parte da variação dos dados.

In [ ]:
#%%

from sklearn.metrics import explained_variance_score

# y_val: valores reais, y_pred: valores previstos
explained_variance = explained_variance_score(y_val, y_pred)

# Exibindo o valor do Explained Variance Score
print(f"Explained Variance Score: {explained_variance}")

Explained Variance Score: 0.7443231548849759


In [ ]:
# %% 

# Após treinar o modelo e validá-lo, podemos salvar o modelo final em um arquivo para uso posterior.
# O arquivo será salvo com o nome 'random_forest.joblib' para fácil carregamento e uso no futuro.
joblib.dump(random_forest, 'random_forest.joblib')

['random_forest.joblib']